<a href="https://colab.research.google.com/github/raqueeb/nlp_bangla/blob/master/bangla_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# আমরা সবসময় চাইবো টেন্সরফ্লো ২.x ব্যবহার করতে
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
import json
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 100000
embedding_dim = 64
max_length = 500
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 350000


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
! pwd

/content


In [ ]:
! ls

gdrive	sample_data


In [ ]:
% cd gdrive

/content/gdrive


In [ ]:
! ls

'My Drive'


In [ ]:
% cd My Drive

/content/gdrive/My Drive


In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
# পারমিশন দেখে নিন
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle datasets download -d furcifer/bangla-newspaper-dataset

bangla-newspaper-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip bangla-newspaper-dataset.zip

Archive:  bangla-newspaper-dataset.zip
  inflating: data/data.json          
  inflating: data_v2/data_v2.json    


In [ ]:
import json
with open('data_v2/data_v2.json', encoding='utf-8') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['content'])
    labels.append(item['category'])

In [ ]:
datastore[0]

{'author': 'গাজীপুর প্রতিনিধি',
 'category': 'bangladesh',
 'category_bn': 'বাংলাদেশ',
 'comment_count': 0,
 'content': 'গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায় আজ বৃহস্পতিবার রাতের টিফিন খেয়ে একটি পোশাক কারখানার ৫০০ শ্রমিক অসুস্থ হয়ে পড়েছেন। এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্সক আল আমিন প্রথম আলো ডটকমকে বলেন, খাদ্যে বিষক্রিয়ায় তাঁরা (শ্রমিকেরা) অসুস্থ হয়ে পড়েছেন। এতে আতঙ্কিত হওয়ার কিছু নেই। অসুস্থদের চিকিত্সা দেওয়া হয়েছে।কারখানার শ্রমিক ও পুলিশ সূত্রে জানা যায়, উপজেলার তেলিরচালা এলাকার সেজাদ সোয়েটার লিমিটেড কারখানার শ্রমিকদের আজ রাত সাড়ে সাতটার দিকে টিফিন দেওয়া হয়। টিফিনে ছিল ডিম, রুটি, পেটিস ও কলা। টিফিন খেয়ে শ্রমিকেরা যথারীতি কাজে যোগ দেন। ওই টিফিন খাওয়ার প্রায় এক ঘণ্টা পর রাত সাড়ে আটটার দিকে কয়েকজন শ্রমিকের বমি ও পেট ব্যথা শুরু হয়। এরপর ধীরে ধীরে পুরো কারখানার শ্রমিকেরা অসুস্থ হতে থাকে। অনেকেই কারখানার মেঝেতে ঢলে পড়ে। এতে পাঁচ শতাধিক শ্রমিক অসুস্থ হয়ে পড়ে।পরে কারখানা কর্তৃপক্ষ দ্রুত যানবাহনের ব্য

In [ ]:
len(sentences)

408471

In [ ]:
set(labels)

{'bangladesh',
 'economy',
 'education',
 'entertainment',
 'international',
 'life-style',
 'opinion',
 'sports',
 'technology'}

In [ ]:
label_mapper = dict([(y,x) for x,y in enumerate(sorted(set(labels)))])
print(label_mapper)
label = np.array( [label_mapper[l] for l in labels] , dtype=np.float32)
print(label[0:5])

{'bangladesh': 0, 'economy': 1, 'education': 2, 'entertainment': 3, 'international': 4, 'life-style': 5, 'opinion': 6, 'sports': 7, 'technology': 8}
[0. 7. 0. 8. 8.]


In [ ]:
# one hot encoded
cl_labels = np.zeros((len(label), len(set(labels))), dtype = np.float32)

for i in range(len(label)):
  cl_labels[ i, int(label[i]) ] = 1.

In [ ]:
cl_labels[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = cl_labels[0:training_size]
testing_labels = cl_labels[training_size:]

In [ ]:
len(testing_sentences)

58471

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
word_index

{'<OOV>': 1,
 'ও': 2,
 'এ': 3,
 'থেকে': 4,
 'করে': 5,
 'করা': 6,
 'বলেন': 7,
 'এই': 8,
 'জন্য': 9,
 'না': 10,
 'তিনি': 11,
 'সঙ্গে': 12,
 'তাঁর': 13,
 'এক': 14,
 'একটি': 15,
 'নিয়ে': 16,
 'এবং': 17,
 'করতে': 18,
 'হয়।': 19,
 'মধ্যে': 20,
 'ওই': 21,
 'আর': 22,
 'এর': 23,
 'পর': 24,
 'না।': 25,
 'গতকাল': 26,
 'প্রথম': 27,
 'বলে': 28,
 'হয়েছে।': 29,
 'কিন্তু': 30,
 'গত': 31,
 'কথা': 32,
 'দুই': 33,
 'হয়ে': 34,
 'সময়': 35,
 'যে': 36,
 'কোনো': 37,
 'তবে': 38,
 'তা': 39,
 'হাজার': 40,
 'হবে।': 41,
 'করার': 42,
 'তাঁদের': 43,
 'হয়': 44,
 'হবে': 45,
 'দিয়ে': 46,
 'করেন': 47,
 'দেওয়া': 48,
 'শুরু': 49,
 'আরও': 50,
 'আগে': 51,
 'পুলিশ': 52,
 'টাকা': 53,
 'ছিল': 54,
 'বিভিন্ন': 55,
 'বা': 56,
 'করেন।': 57,
 'কাজ': 58,
 'বছর': 59,
 'এখন': 60,
 'আজ': 61,
 'তাঁরা': 62,
 'সব': 63,
 'পর্যন্ত': 64,
 'বাংলাদেশ': 65,
 'দিকে': 66,
 'দিন': 67,
 'কর্মকর্তা': 68,
 'হিসেবে': 69,
 'কিছু': 70,
 'জানান': 71,
 'দেখা': 72,
 'জেলা': 73,
 'অনেক': 74,
 'হয়েছে': 75,
 'কাছে': 76,
 'নতুন': 77,
 'উপজেলার': 78,
 'সেই': 79

In [ ]:
training_sequences[0]

[2259,
 15036,
 78,
 72533,
 687,
 61,
 215,
 2489,
 18955,
 8167,
 15,
 1074,
 1368,
 1046,
 620,
 1697,
 294,
 10719,
 3,
 966,
 1270,
 114,
 21,
 1368,
 1,
 17037,
 598,
 1134,
 201,
 8588,
 414,
 1254,
 27,
 795,
 5183,
 7,
 11270,
 57451,
 62,
 2185,
 1697,
 294,
 10719,
 102,
 7279,
 909,
 70,
 150,
 93383,
 8308,
 316,
 1,
 620,
 2,
 52,
 358,
 131,
 721,
 78,
 72533,
 191,
 1,
 39067,
 2420,
 1368,
 1350,
 61,
 253,
 680,
 2990,
 66,
 18955,
 316,
 166,
 81996,
 54,
 3394,
 9610,
 1,
 2,
 81997,
 18955,
 8167,
 2185,
 9313,
 468,
 724,
 240,
 21,
 18955,
 10238,
 692,
 14,
 662,
 24,
 253,
 680,
 1791,
 66,
 1919,
 3591,
 9453,
 2,
 8055,
 3248,
 49,
 166,
 225,
 1768,
 1768,
 806,
 1368,
 2185,
 1697,
 96,
 536,
 1037,
 1368,
 7597,
 9454,
 2517,
 102,
 224,
 1189,
 620,
 1697,
 294,
 1,
 1530,
 760,
 522,
 3950,
 227,
 5,
 94,
 23802,
 727,
 995,
 23802,
 17037,
 995,
 87,
 445,
 40841,
 55,
 5429,
 373,
 116,
 20612,
 10856,
 884,
 6377,
 1,
 2374,
 294,
 1368,
 213,
 1270,


In [ ]:
# টেন্সরফ্লো ২.x এর জন্য এই কনভার্সন

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
training_padded

array([[ 2259, 15036,    78, ...,     0,     0,     0],
       [ 1082,     1, 21406, ...,     0,     0,     0],
       [  764,  2002,   470, ...,     0,     0,     0],
       ...,
       [22488, 39163, 15195, ...,     0,     0,     0],
       [ 1692,   174,  9581, ...,     0,     0,     0],
       [23523, 60309,   558, ...,     0,     0,     0]], dtype=int32)

In [ ]:
# model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#    tf.keras.layers.GlobalAveragePooling1D(),
#    tf.keras.layers.Dense(128, activation='relu'),
#    tf.keras.layers.Dense(9, activation='softmax')
#])

model =  tf.keras.Sequential([
                              tf.keras.layers.Embedding(input_dim=vocab_size, 
                              output_dim=embedding_dim, 
                              input_length=max_length),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
                              tf.keras.layers.Dense(9,  activation='softmax')
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 64)           6400000   
_________________________________________________________________
bidirectional (Bidirectional (None, 500, 256)          197632    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense (Dense)                (None, 9)                 2313      
Total params: 6,994,185
Trainable params: 6,994,185
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 1  
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

In [ ]:
## অ্যাক্যুরেসি এবং লস

import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
## আমি বেশি লুপ ব্যবহার করতে চাইনা বোঝানোর সময়

sentence = ["করোনা নিয়ে অনেক রকম প্রশ্ন আমাদের মনে দেখা দেয় যেমন ঘরের বাইরে সব সময় মাস্ক পরে থাকলে কি অক্সিজেনের অভাব হতে পারে সব সময় দূরত্ব রেখে চললে কি শরীরের রোগ প্রতিরোধক্ষমতা কমে যাবে না"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


In [ ]:
## প্রজেক্টরে দেখানোর জন্য 

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')